#### This notebook is the main notebook to run the recommendations of the procedures based on the lab results and diagnosis

## Load the data

In [1]:
def merge_all(L):
    A = L[0]
    for i in range(1,len(L)):
        A = pd.merge(A, L[i], on='PC_enc', how='inner')
    return A
        

In [2]:
import numpy as np
import pandas as pd



data_test = pd.read_csv('data_processing/data_test.csv')
data_train = pd.read_csv('data_processing/data_train.csv')
#df_test = pd.read_csv('data/data_test.csv').sort_values('PC_enc')

# Get the PC labs and diagnosis
PC_lab_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[13:313])].sort_values('PC_enc')
PC_lab_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[13:313])].sort_values('PC_enc')
print(PC_lab_tr.head())

PC_dx_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[2: 12])].sort_values('PC_enc')
PC_dx_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[2: 12])].sort_values('PC_enc')
print(PC_dx_tr.head())


PC_pr_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[314: 375])].sort_values('PC_enc')
PC_pr_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[314: 375])].sort_values('PC_enc')
print(PC_pr_tr.head())


# privoder information
SP_id_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[377: 407])].sort_values('PC_enc')
SP_id_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[377: 407])].sort_values('PC_enc')




SP_pr_tr = data_train[[data_train.columns[0]]+ list(data_train.columns[407:])].sort_values('PC_enc')
SP_pr_ts = data_test[[data_test.columns[0]]+ list(data_test.columns[407:])].sort_values('PC_enc')
print(SP_pr_tr.head())



# column and 
#Enc_df = PC_dx['PC_enc']
#print(PC_pr.columns)
#Pr_columns = PC_pr.columns[2:]



#X = PC_lab.to_numpy()[:,2:]
#X = X/3.0
#Y_P = SP_pr.to_numpy()[:,2:]
#Z =  PC_dx.to_numpy()[:,2:]
#W = PC_pr.to_numpy()[:,2:]
#Y = Y_P[:,31:]

#print(Z.shape)
df_input_train = merge_all([PC_lab_tr, PC_dx_tr, PC_pr_tr, SP_id_tr])
df_input_test = merge_all([PC_lab_ts, PC_dx_ts, PC_pr_ts, SP_id_ts])

X_train =  df_input_train.to_numpy()[:,1:]
X_test =  df_input_test.to_numpy()[:,1:]

Y_train = SP_pr_tr.to_numpy()[:,1:]
Y_test = SP_pr_ts.to_numpy()[:,1:]


d = X_train.shape[1]
N_train = X_train.shape[0]

d_out = Y_train.shape[1]
print(X_train.shape)
print(Y_train.shape)

        PC_enc  Lab0  Low0  High0  Lab1  Low1  High1  Lab2  Low2  High2  ...  \
4326  16116015     1     0      0     0     0      0     0     0      0  ...   
1525  16175218     1     0      0     1     0      0     1     0      0  ...   
5042  16494386     1     0      0     1     0      0     1     0      0  ...   
5055  16927888     1     0      0     1     0      0     1     0      0  ...   
2418  17091694     1     0      0     1     0      0     1     0      0  ...   

      High96  Lab97  Low97  High97  Lab98  Low98  High98  Lab99  Low99  High99  
4326       0      0      0       0      0      0       0      0      0       0  
1525       0      0      0       0      0      0       0      0      0       0  
5042       0      0      0       0      0      0       0      0      0       0  
5055       0      0      0       0      0      0       0      0      0       0  
2418       0      0      0       0      0      0       0      0      0       0  

[5 rows x 301 columns]
        P

#### We'll start simple, with a single fully-connected neural layer as encoder and as decoder

## Model 1

In [3]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

p = 0.35

input_img = Input(shape=(d,))
encoded = Dense(300, activation='relu')(input_img)
encoded = Dropout(rate=p)(encoded)
encoded = Dense(200, activation='relu')(encoded)
encoded = Dropout(rate=p)(encoded)

encoded = Dense(100, activation='relu')(encoded)
decoded = Dense(80, activation='relu')(encoded)
encoded = Dropout(rate=p)(encoded)
decoded = Dense(d_out, activation='relu')(decoded)

#encoding_dim = 40

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

Using TensorFlow backend.


## Model 2

In [4]:
from keras.layers import Input, Dense
from keras.models import Model


input_img = Input(shape=(d,))
#encoded = Dense(120, activation='relu')(input_img)
#decoded = Dense(40, activation='relu')(encoded)
#decoded = Dense(60, activation='sigmoid')(encoded)

encoded = Dense(40, activation='relu')(input_img)
decoded = Dense(d_out, activation='sigmoid')(encoded)

#encoding_dim = 40

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

#### Now let's train our autoencoder

In [5]:
import keras
opt = keras.optimizers.SGD(lr=5e-2, momentum=0.9, decay=1e-2/100)
autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error') #loss='mean_squared_error')

#### Now let's train our autoencoder for 50 epochs:

In [6]:

autoencoder.fit(X_train, Y_train,
                epochs=300,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, Y_test))#,class_weight=weights) 

Train on 5500 samples, validate on 1321 samples
Epoch 1/300
5500/5500 [==============================] - 0s 22us/step - loss: 0.2430 - val_loss: 0.2438
Epoch 2/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.2421 - val_loss: 0.2430
Epoch 3/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.2412 - val_loss: 0.2422
Epoch 4/300
5500/5500 [==============================] - 0s 12us/step - loss: 0.2403 - val_loss: 0.2414
Epoch 5/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.2394 - val_loss: 0.2405
Epoch 6/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.2385 - val_loss: 0.2397
Epoch 7/300
5500/5500 [==============================] - 0s 14us/step - loss: 0.2376 - val_loss: 0.2389
Epoch 8/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.2366 - val_loss: 0.2380
Epoch 9/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.2356 - val_loss: 0.2371
Epoch 10/300
550

5500/5500 [==============================] - 0s 10us/step - loss: 0.1356 - val_loss: 0.1443
Epoch 79/300
5500/5500 [==============================] - 0s 10us/step - loss: 0.1342 - val_loss: 0.1430
Epoch 80/300
5500/5500 [==============================] - 0s 15us/step - loss: 0.1329 - val_loss: 0.1417
Epoch 81/300
5500/5500 [==============================] - 0s 18us/step - loss: 0.1316 - val_loss: 0.1405
Epoch 82/300
5500/5500 [==============================] - 0s 18us/step - loss: 0.1303 - val_loss: 0.1393
Epoch 83/300
5500/5500 [==============================] - 0s 15us/step - loss: 0.1290 - val_loss: 0.1381
Epoch 84/300
5500/5500 [==============================] - 0s 16us/step - loss: 0.1278 - val_loss: 0.1369
Epoch 85/300
5500/5500 [==============================] - 0s 16us/step - loss: 0.1266 - val_loss: 0.1358
Epoch 86/300
5500/5500 [==============================] - 0s 16us/step - loss: 0.1254 - val_loss: 0.1347
Epoch 87/300
5500/5500 [==============================] - 0s 16us/st

5500/5500 [==============================] - 0s 18us/step - loss: 0.0874 - val_loss: 0.0969
Epoch 156/300
5500/5500 [==============================] - 0s 13us/step - loss: 0.0871 - val_loss: 0.0967
Epoch 157/300
5500/5500 [==============================] - 0s 15us/step - loss: 0.0869 - val_loss: 0.0964
Epoch 158/300
5500/5500 [==============================] - 0s 15us/step - loss: 0.0867 - val_loss: 0.0962
Epoch 159/300
5500/5500 [==============================] - 0s 17us/step - loss: 0.0866 - val_loss: 0.0960
Epoch 160/300
5500/5500 [==============================] - 0s 17us/step - loss: 0.0864 - val_loss: 0.0958
Epoch 161/300
5500/5500 [==============================] - 0s 14us/step - loss: 0.0862 - val_loss: 0.0956
Epoch 162/300
5500/5500 [==============================] - 0s 14us/step - loss: 0.0860 - val_loss: 0.0954
Epoch 163/300
5500/5500 [==============================] - 0s 16us/step - loss: 0.0858 - val_loss: 0.0952
Epoch 164/300
5500/5500 [==============================] - 0

5500/5500 [==============================] - 0s 10us/step - loss: 0.0781 - val_loss: 0.0863
Epoch 233/300
5500/5500 [==============================] - 0s 11us/step - loss: 0.0780 - val_loss: 0.0863
Epoch 234/300
5500/5500 [==============================] - 0s 10us/step - loss: 0.0779 - val_loss: 0.0862
Epoch 235/300
5500/5500 [==============================] - 0s 10us/step - loss: 0.0779 - val_loss: 0.0861
Epoch 236/300
5500/5500 [==============================] - 0s 11us/step - loss: 0.0778 - val_loss: 0.0860
Epoch 237/300
5500/5500 [==============================] - 0s 11us/step - loss: 0.0777 - val_loss: 0.0859
Epoch 238/300
5500/5500 [==============================] - 0s 11us/step - loss: 0.0776 - val_loss: 0.0858
Epoch 239/300
5500/5500 [==============================] - 0s 10us/step - loss: 0.0776 - val_loss: 0.0857
Epoch 240/300
5500/5500 [==============================] - 0s 10us/step - loss: 0.0775 - val_loss: 0.0857
Epoch 241/300
5500/5500 [==============================] - 0

#### After 50 epochs, the autoencoder seems to reach a stable train/test loss value of about 0.11. We can try to visualize the reconstructed inputs and the encoded representations. We will use Matplotlib.

In [7]:

pred = autoencoder.predict(X_test)
pred_train = autoencoder.predict(X_train)

print(pred.shape)


(1321, 61)


### back to dataframe

In [8]:
print(len(SP_pr_ts))


PC_enc_list = SP_pr_ts['PC_enc'].sort_values().tolist()
df_new = pd.DataFrame(data=pred, columns=SP_pr_ts.columns[1:])
print(len(df_new))
df_new['PC_enc'] = PC_enc_list
df_new = df_new[['PC_enc']+ list(df_new.columns[0:-1])]

print(df_new.head())
df_new.to_csv('evaluation/data/Unit_Deep_Model_pred.csv',index=False)



1321
1321
         PC_enc    F0_y_y        F1        F2        F3        F4        F5  \
0  131023122942  0.557230  0.397423  0.115743  0.032196  0.189903  0.195078   
1  131023156693  0.523959  0.366571  0.113077  0.014568  0.163814  0.194907   
2  131023214462  0.497012  0.431876  0.181726  0.031818  0.269040  0.173973   
3  131023214801  0.529826  0.455151  0.438525  0.349678  0.446815  0.462625   
4  131023242322  0.524961  0.398238  0.152882  0.029017  0.209589  0.140291   

         F6        F7        F8  ...     F51_y     F52_y     F53_y     F54_y  \
0  0.085403  0.095224  0.056861  ...  0.037078  0.029743  0.027705  0.032667   
1  0.057891  0.063255  0.038908  ...  0.023304  0.018054  0.015775  0.030632   
2  0.129484  0.087353  0.083867  ...  0.036213  0.044716  0.043409  0.055592   
3  0.382129  0.402601  0.368995  ...  0.331734  0.385043  0.373929  0.338785   
4  0.119025  0.095361  0.061487  ...  0.032200  0.033124  0.033105  0.037803   

      F55_y     F56_y     F57_y   

### OLD CODES

In [29]:
#print(x_test[20,:10])
i = 25
print(pred[i,:])
print(np.argsort(-pred[i,:]))
print(np.where(Y_test[i,:]==1)[0])

[5.75770378e-01 4.32370335e-01 1.99866906e-01 2.53232569e-02
 2.31172889e-01 1.86986595e-01 1.38435274e-01 9.49122608e-02
 5.36040217e-03 1.00482255e-04 1.40880212e-01 1.76444016e-02
 6.84578866e-02 1.38800502e-01 0.00000000e+00 1.04437999e-01
 8.47560614e-02 1.08890623e-01 0.00000000e+00 1.21328374e-02
 0.00000000e+00 9.65663716e-02 1.53759904e-02 2.90494207e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.52837127e-01 8.20719898e-02 9.88068804e-03 1.72577128e-02
 1.41113102e-02 0.00000000e+00 3.05181332e-02 1.72735620e-02
 0.00000000e+00 0.00000000e+00 5.29543310e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.63979826e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.96504253e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.21714865e-03 0.00000000e+00
 3.66613944e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
[ 0  1  4  2  5 28 10 13  6 17 15 21  7 16 29 12 38 50 34 23 46  3 1

In [43]:
print(np.mean(y_test,axis=0))
print(np.mean(pred,axis=0))

[0.56284658 0.43715342 0.21072089 0.00924214 0.24399261 0.1922366
 0.1284658  0.12107209 0.12939002 0.07486137 0.16358595 0.05822551
 0.08225508 0.08780037 0.06099815 0.10813309 0.04990758 0.09704251
 0.04251386 0.03789279 0.04158965 0.07855823 0.025878   0.08595194
 0.0194085  0.0323475  0.03327172 0.01571165 0.11460259 0.11460259
 0.01848429 0.05360444 0.02495379 0.02310536 0.04343808 0.02310536
 0.04066543 0.02402957 0.01756007 0.11460259 0.01571165 0.00554529
 0.02680222 0.01478743 0.01478743 0.01201479 0.00831793 0.02033272
 0.03142329 0.01201479 0.05360444 0.00369686 0.02402957 0.00739372
 0.01756007 0.0064695  0.02218115 0.01848429 0.02125693 0.0064695
 0.0064695 ]
[0.7230316  0.500941   0.328649   0.1599551  0.28746173 0.3005466
 0.31592745 0.24927387 0.13052641 0.1918064  0.22012499 0.21638663
 0.25570133 0.312817   0.22577952 0.21150313 0.20382686 0.32516876
 0.14882538 0.13547121 0.19084738 0.09776655 0.14411257 0.15672007
 0.2337083  0.11784337 0.16430606 0.20043522 0.14219

### Find Jackard Similarity

In [30]:
p=5
J = np.zeros(100)
for i in range(100):
    A = set(np.argsort(-pred[i,:])[:p])
    B = set(np.where(Y_test[i,:]==1)[0])
    J[i] = len(A.intersection(B))/p #/len(A.union(B))
    print(A,B)

print(np.mean(J))

{0, 1, 2, 4, 5} {2, 10, 13}
{0, 1, 2, 4, 5} {9, 4, 6, 25}
{0, 1, 2, 4, 5} {2, 19, 21, 6}
{0, 1, 2, 4, 5} {17, 4, 5, 7}
{0, 1, 2, 4, 10} {0, 1, 15}
{0, 1, 2, 4, 10} set()
{0, 1, 2, 4, 5} set()
{0, 1, 2, 4, 5} {0, 12, 50, 56, 58}
{0, 1, 2, 4, 10} {0, 17, 5}
{0, 1, 2, 4, 5} {0, 8, 2, 30}
{0, 1, 2, 4, 5} {11, 23, 7}
{0, 1, 2, 4, 28} {34, 3, 35, 6, 9, 10, 13, 19, 25}
{0, 1, 2, 4, 5} {0, 1, 39, 8, 15, 29}
{0, 1, 2, 4, 5} {0, 1, 39, 8, 15, 29}
{0, 1, 2, 4, 5} {0}
{0, 1, 2, 4, 28} {48}
{0, 1, 4, 5, 6} {0, 1, 32, 33, 4, 35, 7, 8, 39, 11, 14, 15, 16, 18, 23, 29}
{0, 1, 2, 4, 5} {48}
{0, 1, 2, 4, 10} {34, 4, 6, 9, 13, 19, 21, 25}
{0, 1, 2, 4, 28} {30}
{0, 1, 4, 5, 21} {5}
{0, 1, 2, 4, 5} {0, 1, 35, 14, 18, 23}
{0, 1, 2, 4, 5} {8, 48, 29, 39}
{0, 1, 2, 4, 5} {0, 1}
{0, 1, 2, 4, 5} {0, 2, 4, 36, 6, 7, 8, 37, 42, 16, 52, 28}
{0, 1, 2, 4, 5} {11}
{0, 1, 2, 4, 10} {0, 2, 29, 39}
{0, 1, 2, 4, 5} {0, 1, 39, 8, 15, 29}
{0, 1, 2, 4, 7} {16, 10, 7}
{0, 1, 2, 4, 5} {4, 5}
{0, 1, 4, 5, 10} {0, 1, 39, 8, 29}


### probability threshold

In [32]:
#print( set(pred[i,np.where(pred[25,:]>0.2)[0]]))

J = np.zeros(1000)
for i in range(1000):
    A = set(np.where(pred[i,:]>0.3)[0])
    B = set(np.where(Y_test[i,:]==1)[0])
    J[i] = len(A.intersection(B))/len(A) #/len(A.union(B))

print(np.mean(J))

0.526


### Check the percentage of the labels

In [50]:
print(np.mean(y_test,axis=0))
print(np.mean(pred,axis=0))

[0.56284658 0.43715342 0.21072089 0.00924214 0.24399261 0.1922366
 0.1284658  0.12107209 0.12939002 0.07486137 0.16358595 0.05822551
 0.08225508 0.08780037 0.06099815 0.10813309 0.04990758 0.09704251
 0.04251386 0.03789279 0.04158965 0.07855823 0.025878   0.08595194
 0.0194085  0.0323475  0.03327172 0.01571165 0.11460259 0.11460259
 0.01848429 0.05360444 0.02495379 0.02310536 0.04343808 0.02310536
 0.04066543 0.02402957 0.01756007 0.11460259 0.01571165 0.00554529
 0.02680222 0.01478743 0.01478743 0.01201479 0.00831793 0.02033272
 0.03142329 0.01201479 0.05360444 0.00369686 0.02402957 0.00739372
 0.01756007 0.0064695  0.02218115 0.01848429 0.02125693 0.0064695
 0.0064695 ]
[0.7230316  0.500941   0.328649   0.1599551  0.28746173 0.3005466
 0.31592745 0.24927387 0.13052641 0.1918064  0.22012499 0.21638663
 0.25570133 0.312817   0.22577952 0.21150313 0.20382686 0.32516876
 0.14882538 0.13547121 0.19084738 0.09776655 0.14411257 0.15672007
 0.2337083  0.11784337 0.16430606 0.20043522 0.14219

In [51]:
print(np.where(pred[:,3]>0.3)[0].shape[0]/pred.shape[0])
print(np.mean(pred[:,3]))
print(np.std(pred[:,3]))

0.011090573012939002
0.1599551
0.033828806


### Print some of the predictions

In [57]:
J = np.zeros(100)
for i in range(100):
    A = set(np.where(pred[i,:]>0.4)[0])
    B = set(np.where(y_test[i,:]==1)[0])
    J[i] = len(A.intersection(B))/len(A) #/len(A.union(B))
    print(A,B) #, pred[i,np.array(list(B))])



{0, 1} {0, 1, 2}
{0, 1} {0, 1, 39, 8, 12, 44, 29, 31}
{0, 1} {0, 1}
{0, 1} {4}
{0, 1, 2, 5, 6, 7, 9, 13, 17, 53, 27} {0, 1, 4, 6, 8, 9, 14, 49, 18, 19, 20, 21, 23, 25, 27}
{0, 1} {6}
{0, 1} {48}
{0, 1} {16, 32, 23, 7}
{0, 1} {0, 5}
{0, 1} {0, 1, 31}
{0, 1} {48}
{0, 1} {0, 5, 39, 40, 29}
{0, 1} {0, 1, 10, 28}
{0, 1} {2, 4, 6, 9, 13, 22}
{0, 1} {12}
{0, 1} {0, 1}
{0, 1, 6, 13, 60} {0, 1, 33, 34, 14, 20, 21}
{0, 1} {0, 1, 5, 39, 29}
{0, 1} {0, 1, 26}
{0, 1} {23}
{0, 1} {0, 1, 2, 4}
{0, 1} {21}
{0, 1} {0, 4, 5, 10, 17, 28}
{0, 1} {2}
{0, 1} {0, 4, 5, 17, 31}
{0, 1} {0, 1, 10, 12, 26}
{0, 1, 6} {0, 1, 35, 49}
{0, 1} set()
{0, 1} {0, 4}
{0, 1} {0, 5, 10, 11, 43, 17, 28}
{0, 1} {0}
{0, 1} {21, 4, 5, 52}
{0, 1} {16, 4, 5, 7}
{0, 1} {22}
{0, 1, 6} {0, 1, 39, 40, 29}
{0, 1} {0}
{0, 1} {0, 1, 4, 5, 13}
{0, 1} {0, 1, 39, 8, 12, 15, 29}
{0, 1, 6} {0, 1, 2, 42, 14}
{0, 1} {0, 1}
{0, 1, 2} {0, 1, 2, 4, 6, 9, 20}
{0, 1} {0, 4, 31}
{0, 1} {2, 5, 22}
{0, 1} {0, 34, 8, 14, 20, 26}
{0, 1, 6, 13, 17, 24, 6

In [55]:
A = {1,2}
print(np.array(list(A)))

[1 2]
